In [1]:
import osmnx as ox
import networkx as nx
import matplotlib.cm as cm
import matplotlib.colors as colors
import pandas as pd
import datetime
import geocoder
import os
from apiclient import discovery
from httplib2 import Http
import oauth2client
import io
from googleapiclient.http import MediaIoBaseDownload
from oauth2client import file, client, tools
%matplotlib inline
ox.config(log_console=True, use_cache=True)

Setting up Google Drive API [Tutorial](https://medium.com/@umdfirecoml/a-step-by-step-guide-on-how-to-download-your-google-drive-data-to-your-jupyter-notebook-using-the-52f4ce63c66c)

In [3]:
obj = lambda: None
lmao = {"auth_host_name":'localhost', 'noauth_local_webserver':'store_true', 'auth_host_port':[8080, 8090], 'logging_level':'ERROR'}
for k, v in lmao.items():
    setattr(obj, k, v)
    
# authorization boilerplate code
SCOPES = 'https://www.googleapis.com/auth/drive.readonly'
store = file.Storage('token.json')
creds = store.get()
# The following will give you a link if token.json does not exist, the link allows the user to give this app permission
if not creds or creds.invalid:
    flow = client.flow_from_clientsecrets('client_id.json', SCOPES)
    creds = tools.run_flow(flow, store, obj)


Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?client_id=656728414845-jtv4fjijjqd5eoocrq67htttitb5qof2.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.readonly&access_type=offline&response_type=code

Enter verification code: 4/KQFyB2diVzcr2rcLcwK5PuG7kND91lCR2LvdzaosSM_a1uw56wDzhzQ
Authentication successful.


In [4]:
cities = ['Batroun', 'Cap-Haitien', 'Lima', 'Montevideo', 'Province of Palawan', 'Sfax', 'Sousse', 'Vinh Long']

In [5]:
def list_ids(response):
    d={}
    page_token =None
    
    while True:
        for file in response.get('files', []):
            d[file.get('name')] = file.get('id')
            print ('Found file: %s (%s)' % (file.get('name'), file.get('id')))
        page_token = response.get('nextPageToken', None)
        if page_token is None:
            break

    
    return d

In [23]:
def get_files_from_drive():
    d1={}
    DRIVE = discovery.build('drive', 'v3', http=creds.authorize(Http()))
    page_token = None
    ##
    # To access data from 'shared with me' folder, replace q with 
    #"sharedWithMe and name= = '{city}' and mimeType = 'application/vnd.google-apps.folder'".format(city=i)
    ##
    for i in cities:
        response = DRIVE.files().list(q="name = '{city}' and mimeType = 'application/vnd.google-apps.folder'".format(city=i),
                                              spaces='drive',
                                              fields='nextPageToken, files(id, name)',
                                              pageToken=page_token).execute()

        di = list_ids(response)    
        print('di', di ,'\n')

        response_ = DRIVE.files().list(q="'{}' in parents".format(di['{}'.format(i)]),
                                              spaces='drive',
                                              fields='nextPageToken, files(id, name)',
                                              pageToken=page_token).execute()
        d_ = list_ids(response_)
        print('d_', d_ ,'\n')
        for j in d_.keys():
            file_id = d_[j]
            request = DRIVE.files().get_media(fileId=file_id)
            # replace the filename and extension in the first field below
            try:
                os.mkdir('drive_data/{}'.format(i))
            except FileExistsError:
                pass
            except FileNotFoundError:
                os.mkdir('drive_data')

            s = j.split('.')
            try:
                ext=s[1]
            except IndexError:
                ext=None

            if ext is None and '_g' not in j:
                try:
                    os.mkdir('drive_data/{city}/{base}'.format(city=i, base=j))
                except FileExistsError:
                    pass
            else:
                try:
                    fh = io.FileIO('drive_data/{city}/{file}.{ext}'.format(city=i, file=s[0], ext=ext), mode='w')
                except:
                    fh = io.FileIO('drive_data/{city}/{file}'.format(city=i, file=j), mode='w')
            downloader = MediaIoBaseDownload(fh, request)
            done = False
            while done is False:
                try:
                    status, done = downloader.next_chunk()
                    print("Download %d%%." % int(status.progress() * 100))
                except:
                    response_1 = DRIVE.files().list(q="'{}' in parents".format(d_[j]),
                                              spaces='drive',
                                              fields='nextPageToken, files(id, name)',
                                              pageToken=page_token).execute()
                    d1 = list_ids(response_1)
                    print('d1', d1, '\n')

                    for k in d1.keys():
                        file_id = d1[k]
                        request = DRIVE.files().get_media(fileId=file_id)
                        # replace the filename and extension in the first field below
                        try:
                            os.mkdir('drive_data/{city}/{base}'.format(city=i, base=j))
                        except FileExistsError:
                            pass
                        
                        t = k.split('.')
                        try:
                            ext=s[1]
                        except IndexError:
                            ext=None
                            
                        if ext is not None:
                            fh = io.FileIO('drive_data/{city}/{base}/{child}.{ext}'.format(city=i, base=j, child=k[:-4], ext=ext), mode='w')
                        else:
                            fh = io.FileIO('drive_data/{city}/{base}/{child}'.format(city=i, base=j, child=k), mode='w')
                        downloader = MediaIoBaseDownload(fh, request)
                        done = False
                        while done is False:
                            status, done = downloader.next_chunk()
                            print("Download %d%%." % int(status.progress() * 100))


In [24]:
get_files_from_drive()

Found file: Batroun (1CnlITgyySGSFrU409ZvjzwzJFdgHHdpf)
di {'Batroun': '1CnlITgyySGSFrU409ZvjzwzJFdgHHdpf'} 

Found file: Batroun.png (1XBjxAKJM-m9W7kNFd6ug5AcqqNTe7LRE)
Found file: Batroun_graph (1XcsXErC9m-sEwnciBHTpWdi-064HImVe)
Found file: Basic_Stats_Batroun.csv (1L75MvA3bexeel3CInFJryhRZeptUWOs1)
Found file: Extended_Stats_Batroun.csv (19V7-Aber_KSuIHa592Q74DthYjw4bbTh)
Found file: nodes (1L2RXXqstiLWBL4EqBd-gBb9AzBtpv6yn)
Found file: Batroun_boundary (1ha1gC3Ix85aMO3Hgrdyh7Rr4NRffAoxy)
Found file: edges (1Qy7LqMdHASFu25-s6eD7gzkIuDGgt4dt)
d_ {'Batroun.png': '1XBjxAKJM-m9W7kNFd6ug5AcqqNTe7LRE', 'Batroun_graph': '1XcsXErC9m-sEwnciBHTpWdi-064HImVe', 'Basic_Stats_Batroun.csv': '1L75MvA3bexeel3CInFJryhRZeptUWOs1', 'Extended_Stats_Batroun.csv': '19V7-Aber_KSuIHa592Q74DthYjw4bbTh', 'nodes': '1L2RXXqstiLWBL4EqBd-gBb9AzBtpv6yn', 'Batroun_boundary': '1ha1gC3Ix85aMO3Hgrdyh7Rr4NRffAoxy', 'edges': '1Qy7LqMdHASFu25-s6eD7gzkIuDGgt4dt'} 

Download 100%.
Download 100%.
Download 100%.
Download 10

Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Found file: edges.shp (1rBWCklxzZ6qK8VIwJSmvh5sHtEROKcKD)
Found file: edges.dbf (1wQN-R_Xi-meKYYSEU1tcTUKsrHaryX7a)
Found file: edges.shx (1OW5fC537vCg6U2MT23_T9ggrhuctX7O-)
Found file: edges.prj (1Rf7H4gxJ4KosW7dsae0_pMScOJW6MS5w)
Found file: edges.cpg (1S6ZaciEvJr-MhzxNoEJQHmNdVBii8X0u)
d1 {'edges.shp': '1rBWCklxzZ6qK8VIwJSmvh5sHtEROKcKD', 'edges.dbf': '1wQN-R_Xi-meKYYSEU1tcTUKsrHaryX7a', 'edges.shx': '1OW5fC537vCg6U2MT23_T9ggrhuctX7O-', 'edges.prj': '1Rf7H4gxJ4KosW7dsae0_pMScOJW6MS5w', 'edges.cpg': '1S6ZaciEvJr-MhzxNoEJQHmNdVBii8X0u'} 

Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Found file: Province of Palawan (13XqIe2zwwkuMSXrdu8oyvMYq8Ad9nB1S)
di {'Province of Palawan': '13XqIe2zwwkuMSXrdu8oyvMYq8Ad9nB1S'} 

Found file: Province of Palawan_graph (11bltqckNLqtM1LhfosqpgNtT2KelEg9W)
Found file: Extended_Stats_Province of Palawan.csv (13lE7XZitczQLTnCLhV3s8j28TiVPnKJf)
Found fil

Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Found file: Vinh Long (1rMh4i1t5sMPaVEtP5LYwhiZyY39TtsVb)
di {'Vinh Long': '1rMh4i1t5sMPaVEtP5LYwhiZyY39TtsVb'} 

Found file: Vinh Long_graph (1urLRzGHdv3J_QmAEVTfGdVkkOKppCMu-)
Found file: Extended_Stats_Vinh Long.csv (1YkoE_PI51XVgCM9WKiSdtREkxuztopMN)
Found file: Basic_Stats_Vinh Long.csv (1qDRiuVLXyRbBaoxW6-cOdwbmLkY-R5YB)
Found file: edges (1gcpjWIWMv3W8l6fMz7GoiPL3bNHGsw4j)
Found file: nodes (1-ztwZOvSbK95fupCs1N-I3mu39gI7hSl)
d_ {'Vinh Long_graph': '1urLRzGHdv3J_QmAEVTfGdVkkOKppCMu-', 'Extended_Stats_Vinh Long.csv': '1YkoE_PI51XVgCM9WKiSdtREkxuztopMN', 'Basic_Stats_Vinh Long.csv': '1qDRiuVLXyRbBaoxW6-cOdwbmLkY-R5YB', 'edges': '1gcpjWIWMv3W8l6fMz7GoiPL3bNHGsw4j', 'nodes': '1-ztwZOvSbK95fupCs1N-I3mu39gI7hSl'} 

Download 100%.
Download 100%.
Download 100%.
Found file: edges.dbf (1iSulgfPlu48QFvWa30MrmzE825iCAdPt)
Found file: edges.shx (1EKx9IFsBWQgjQztKc0kXaxWpXU87t2eG)
Found file: edges.prj (1Zflzj3m02CFJAg

### Read in the edges data with edge_centrality

In [21]:
import geopandas as gpd
import pylab as pl
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [25]:
lima_edges = gpd.GeoDataFrame.from_file('drive_data/Lima/edges/edges.shp')

In [26]:
lima_edges.head()

,access,bridge,edge_centr,from,highway,junction,key,lanes,length,maxspeed,name,oneway,osmid,ref,to,tunnel,geometry
0,None,None,0.0013667980614227517,1475218848,residential,None,0,None,60.413,30,JirÃ³n Ricardo Rivera Navarrete,False,111888580,None,1273610289,None,LINESTRING (273650.9493054305 -1333416.5851088...
1,None,None,0.0021078431725473066,1273734897,tertiary,None,0,2,59.608,40,Mariano Arredondo,False,420679867,None,1273610289,None,LINESTRING (273734.7238754003 -1333396.6634348...
2,None,None,0.0017321081556805808,1397486926,tertiary,None,0,2,49.814,40,Mariano Arredondo,False,420679867,None,1273610289,None,LINESTRING (273642.1128754931 -1333338.5098243...
3,None,None,8.931754046724511e-05,1397486923,residential,None,0,None,80.844,30,JirÃ³n Ricardo Rivera Navarrete,False,111888580,None,1273610289,None,LINESTRING (273741.5749159084 -1333323.4596627...
4,None,None,0.0016878210116633775,1624735796,residential,None,0,None,196.227,30,Calle 11,False,149509214,None,1624735795,None,LINESTRING (274169.4528293905 -1334164.6677177...


### Read in the graph data

In [27]:
G = nx.read_gpickle("drive_data/Lima/Lima_graph.None")

In [35]:
edgelist = nx.to_pandas_edgelist(G)

In [36]:
edgelist.head()

,source,target,length,maxspeed,name,ref,highway,oneway,junction,geometry,access,bridge,lanes,osmid,tunnel
0,1273610289,1475218848,60.413,30,Jirón Ricardo Rivera Navarrete,NaN,residential,False,NaN,NaN,NaN,NaN,NaN,111888580,NaN
1,1273610289,1273734897,59.608,40,Mariano Arredondo,NaN,tertiary,False,NaN,NaN,NaN,NaN,2,420679867,NaN
2,1273610289,1397486926,49.814,40,Mariano Arredondo,NaN,tertiary,False,NaN,NaN,NaN,NaN,2,420679867,NaN
3,1273610289,1397486923,80.844,30,Jirón Ricardo Rivera Navarrete,NaN,residential,False,NaN,LINESTRING (273683.6306425154 -1333365.9907826...,NaN,NaN,NaN,111888580,NaN
4,1624735795,1624735796,196.227,30,Calle 11,NaN,residential,False,NaN,LINESTRING (274125.0929517995 -1333974.4616061...,NaN,NaN,NaN,149509214,NaN
